In [ ]:
# default_exp prot_bert

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export
from transformers import BertForMaskedLM, BertTokenizer, pipeline
import pandas as pd

In [ ]:
# export
from berteome import berteomeDF

In [ ]:
# export
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertForMaskedLM.from_pretrained("Rostlab/prot_bert")
unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# export
def spacifySeq(seq):
  return ' '.join(list(seq))

In [ ]:
assert spacifySeq("MENDEL") == 'M E N D E L'

In [ ]:
' '.join(list("MENDEL"))

'M E N D E L'

In [ ]:
# export 
def maskifySeq(seq, pos, mask="[MASK]"):
  spacifiedSeq = spacifySeq(seq)
  seqList = spacifiedSeq.split()
  seqList[pos] = mask
  return " ".join(seqList)

In [ ]:
assert maskifySeq("MENDEL", 3) == 'M E N [MASK] E L'

NameError: name 'maskifySeq' is not defined

In [ ]:
# export
def allResidueCoordinates(seq,residue):
  return [i for i, x in enumerate(seq) if x == residue]

This should be renamed, to something along the lines of bertPredictionDict

In [ ]:
# export
def aaPosDict(aas):
    aaDict = {}
    for aaPos in range(len(aas)):
        aa = aas[aaPos]
        aaDict[aa] = aaPos
    return aaDict

In [ ]:
aaPosDict("ACDEFGHIKLMNPQRSTVWY")

{'A': 0,
 'C': 1,
 'D': 2,
 'E': 3,
 'F': 4,
 'G': 5,
 'H': 6,
 'I': 7,
 'K': 8,
 'L': 9,
 'M': 10,
 'N': 11,
 'P': 12,
 'Q': 13,
 'R': 14,
 'S': 15,
 'T': 16,
 'V': 17,
 'W': 18,
 'Y': 19}

In [ ]:
# export 
def bertPredictionDF(seq, aas="ACDEFGHIKLMNPQRSTVWY"):
  aaDict = aaPosDict(aas)
  bertPredDict = {}
  # posPredictions = []
  for aaPos in range(len(seq)):
    aa = seq[aaPos]
    maskPosSeq = maskifySeq(seq, aaPos)
    predictions = unmasker(maskPosSeq, top_k=30)
    predList = [0]*len(aas)
    for prediction in predictions:
      predAA = prediction["token_str"]
      if predAA in aaDict:
        predList[aaDict[predAA]] = prediction["score"]
    bertPredDict[aaPos] = predList
  bertPredDF = berteomeDF.modelPredDF(bertPredDict,seq, aas).predDf
  return bertPredDF

In [ ]:
bertPredictionDF("MENDEL")

,wt,wtIndex,A,C,D,E,F,G,H,I,...,M,N,P,Q,R,S,T,V,W,Y
0,M,1,0.036685,0.011501,0.048229,0.118868,0.024064,0.039190,0.012617,0.066477,...,0.076580,0.072637,0.024714,0.038660,0.043091,0.070257,0.056526,0.049911,0.007779,0.021692
1,E,2,0.045712,0.015659,0.041913,0.074816,0.037146,0.044317,0.018260,0.073063,...,0.043572,0.062655,0.025272,0.036905,0.055532,0.064412,0.049945,0.056779,0.012689,0.029887
2,N,3,0.043558,0.009684,0.162566,0.184336,0.033777,0.044654,0.012353,0.052622,...,0.041478,0.041984,0.019989,0.025511,0.029428,0.048098,0.030299,0.054734,0.007428,0.024920
3,D,4,0.042079,0.013243,0.049744,0.086189,0.039733,0.055907,0.016860,0.073291,...,0.040078,0.060817,0.032022,0.039686,0.046224,0.062319,0.044898,0.058933,0.010875,0.026594
4,E,5,0.046638,0.018769,0.079816,0.086908,0.050634,0.050462,0.022395,0.074495,...,0.028960,0.062229,0.023877,0.030532,0.040486,0.065190,0.044934,0.068032,0.012155,0.038031
5,L,6,0.035695,0.008615,0.060928,0.142576,0.019581,0.046287,0.013043,0.060374,...,0.037424,0.090177,0.019358,0.032733,0.043823,0.045863,0.043224,0.045121,0.009800,0.021241


I should probably think about this more, maybe I should go ahead and put the data into the dataframe format that I want from here. It probably doesn't make too much sense to make this structure (which has little utility) just to make the useful structure. Instead, I think I should go ahead and start iterating through predictions and add those predictions to a dict that will be easily converted to a dataframe.

I think I would just need to know the intended index for the 20 amino acids, since the order of the predicitons is sorted by score